In [ ]:
# 유사앙상블 코드
# 모델의 낮은 아큐러시를 보완하기 위하여 한 유저의 이미지를 여러개 받아서 그에 따른 여러개의 프레딕트 값의 평균을 계산하여 진단을 내린다. 앙상블과 유사한
# 효과를 내기 위한 방법이다. 프레딕트 값을 전체데이터의 수로 나누어 누적시킨다

구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install efficientnet_pytorch

import torchvision
from torchvision import transforms
import os
from torch.utils.data import Dataset,DataLoader
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=1d63ea5299b726247b143b22aee48504ad629025d5ed6551c822cb00d5a7bb4c
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
#웹페이지에서 이미지를 업로드 받은 후 연산시간을 줄이기 위해 미리 로드하는 것
#PATH
PATH1 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model1.pt'  # 모델1
PATH2 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model2.pt'  # 모델2
PATH3 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model3.pt'  # 모델3
PATH4 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model4.pt'  # 모델4
PATH5 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model5.pt'  # 모델5
PATH6 = '/content/drive/MyDrive/project/scalp_weights/'+'aram_model6.pt'  # 모델6
#cuda # 쿠다를 쓸 수 있다면 쿠다 아니면 시피유        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model load        
model1 = torch.load(PATH1, map_location=device)
model2 = torch.load(PATH2, map_location=device)
model3 = torch.load(PATH3, map_location=device)
model4 = torch.load(PATH4, map_location=device)
model5 = torch.load(PATH5, map_location=device)
model6 = torch.load(PATH6, map_location=device)
#아웃풋, 로스, 프레딕, 아큐러시
#모델 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
model1.eval() 
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()
# 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
transforms_test = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])

In [ ]:
# root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/sample/ensemble' ,
                    transform = transforms_test)

# DataLoader를 통해 네트워크에 올리기
from torch.utils.data import Dataset,DataLoader 
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
#다중이미지를 받아서 평균프레딕트값을 구할 변수
fm1p=0.0
fm2p=0.0
fm3p=0.0
fm4p=0.0
fm5p=0.0
fm6p=0.0
##아웃풋, 로스, 프레딕, 아큐러시
#output_list = []
#correct = 0 # 그래프그리기위한
#로스 연산을 위한
#import torch.nn.functional as F   # F : 테스트_로스 연산 함수
#test_loss = 0 # 로스연산을위한 
from tqdm import tqdm # 진행률 표시를 위한
if __name__ == '__main__':
    with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
            for data, target in tqdm(testloader):                                   
                data    = data.to(device)             
                output1 = model1(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                output2 = model2(data) 
                output3 = model3(data) 
                output4 = model4(data) 
                output5 = model5(data) 
                output6 = model6(data)  
#predict # # 0~3값만 뽑기 
                m1p = output1.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm1p += m1p / len(testloader.dataset)
                m2p = output2.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm2p += m2p / len(testloader.dataset)
                m3p = output3.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm3p += m3p / len(testloader.dataset)
                m4p = output4.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm4p += m4p / len(testloader.dataset)
                m5p = output5.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm5p += m5p / len(testloader.dataset)
                m6p = output6.argmax(dim=1, keepdim=True)[0][0].tolist() # argmax로 최댓값의 인덱스 뽑기
                fm6p += m6p / len(testloader.dataset)
print()
print()
print(fm1p)
print(fm2p)
print(fm3p)
print(fm4p)
print(fm5p)
print(fm6p)
print()
#진단
d_list = []     # 두피유형진단결과
#두피 유형 진단법                          # ==0 은 1보다 작다로 변환, !=0은 1이상으로 변환
if fm1p < 1  and fm2p < 1 and fm3p < 1 and fm4p < 1 and fm5p < 1 and fm6p < 1 :
    d1 = '정상입니다.'
    d_list.append(d1)
elif fm1p >= 1 and fm2p < 1 and fm3p < 1 and fm4p < 1 and fm5p < 1 and fm6p < 1 :
    d2 = '건성 두피입니다.' 
    d_list.append(d2)
elif fm1p < 1 and fm2p >=1  and fm3p < 1 and fm4p < 1 and fm5p < 1 and fm6p < 1 :
    d3 = '지성 두피입니다.'
    d_list.append(d3)
elif fm2p < 1 and fm3p >= 1 and fm4p < 1 and fm5p < 1  and fm6p < 1 :
    d4 = '민감성 두피입니다.'
    d_list.append(d4)
elif fm2p >= 1 and fm3p >= 1 and fm4p < 1 and fm6p < 1 :
    d5 = '지루성 두피입니다.'
    d_list.append(d5)
elif fm3p < 1 and fm4p  >= 1 and fm6p < 1 :
    d6 = '염증성 두피입니다.'
    d_list.append(d6)
elif fm3p < 1 and fm4p < 1 and fm5p >= 1 and fm6p < 1 :
    d7 = '비듬성 두피입니다.'
    d_list.append(d7)
elif fm1p < 1 and fm2p >= 1 and fm3p < 1 and fm4p < 1 and fm5p < 1 and fm6p >= 1 :
    d8 = '탈모입니다.'
    d_list.append(d8)
else:
    d9 = '복합성 두피입니다.'
    d_list.append(d9)
print(d_list[0])
                #output_list.append(output);
                #test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                #pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                #correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  pred.eq(data) : pred와 data가 일치하는지 검사
#test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기
#아큐러시 출력 ( :.4f 소수점반올림 )
#print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값
#로스, 아큐러시 출력
#print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))

100%|██████████| 119/119 [01:10<00:00,  1.69it/s]



1.1512605042016801
1.1848739495798317
1.487394957983192
0.8991596638655458
0.8487394957983188
2.2689075630252065

복합성 두피입니다.
